In [1]:
import numpy as np
import pandas as pd

In [20]:
import pandas as pd
import numpy as np

np.random.seed(42)

# Define grade rules
grade_criteria = {
    "A": {
        "AA Level 1": (0, 0),
        "AA Level 2": (0, 0),
        "AA Level 3": (0, 0),
        "A Level 1": (0, 3),
        "A Level 2": (0, 2),
        "A Level 3": (0, 0),
        "B Level 1": (0, 4),
        "B Level 2": (0, 3),
        "B Level 3": (0, 1),
    },
    "B": {
        "AA Level 1": (0, 5),
        "AA Level 2": (0, 2),
        "AA Level 3": (0, 2),
        "A Level 1": (0, 10),
        "A Level 2": (0, 5),
        "A Level 3": (0, 3),
        "B Level 1": (0, 20),
        "B Level 2": (0, 10),
        "B Level 3": (0, 5),
    },
    "C": {
        "AA Level 1": (0, 50),
        "AA Level 2": (0, 20),
        "AA Level 3": (0, 10),
        "A Level 1": (0, 50),
        "A Level 2": (0, 50),
        "A Level 3": (0, 20),
        "B Level 1": (0, 50),
        "B Level 2": (0, 50),
        "B Level 3": (0, 50),
    },
}

# Defect columns
defect_columns = list(grade_criteria["A"].keys())

# Grading function based on defined criteria
def assign_grade_strict(row):
    def within(row, limits):
        return all(limits[col][0] <= row[col] <= limits[col][1] for col in defect_columns)

    if within(row, grade_criteria["A"]):
        return "A"
    elif within(row, grade_criteria["B"]):
        return "B"
    elif within(row, grade_criteria["C"]):
        return "C"
    else:
        return "D"

# Generate samples by grade with exact limits
def generate_strict_grade_samples(grade_label, limits, desired_count):
    samples = []
    attempts = 0
    max_attempts = 50000
    while len(samples) < desired_count and attempts < max_attempts:
        row = {col: np.random.randint(low, high + 1) for col, (low, high) in limits.items()}
        row_series = pd.Series(row)
        grade = assign_grade_strict(row_series)
        if grade == grade_label:
            row["Grade"] = grade
            samples.append(row)
        attempts += 1
    return samples

# Generate 250 samples for A, B, and C
samples_A = generate_strict_grade_samples("A", grade_criteria["A"], 250)
samples_B = generate_strict_grade_samples("B", grade_criteria["B"], 250)
samples_C = generate_strict_grade_samples("C", grade_criteria["C"], 250)

# Generate 250 samples for Grade D (values intentionally beyond Grade C)
samples_D = []
while len(samples_D) < 250:
    row = {col: np.random.randint(51, 101) for col in defect_columns}
    if assign_grade_strict(pd.Series(row)) == "D":
        row["Grade"] = "D"
        samples_D.append(row)

# Combine all
final_df = pd.DataFrame(samples_A + samples_B + samples_C + samples_D)

# Save to CSV if needed
final_df.to_csv("strictly_balanced_mobile_defect_dataset.csv", index=False)

# Print a sample
print(final_df.head())


   AA Level 1  AA Level 2  AA Level 3  A Level 1  A Level 2  A Level 3  \
0           0           0           0          2          0          0   
1           0           0           0          0          2          0   
2           0           0           0          2          2          0   
3           0           0           0          3          2          0   
4           0           0           0          3          1          0   

   B Level 1  B Level 2  B Level 3 Grade  
0          2          3          0     A  
1          1          2          0     A  
2          4          3          1     A  
3          4          1          1     A  
4          1          3          1     A  


In [25]:
final_df[final_df['Grade'] == 'D']


,AA Level 1,AA Level 2,AA Level 3,A Level 1,A Level 2,A Level 3,B Level 1,B Level 2,B Level 3,Grade
750,75,70,99,72,71,83,88,65,96,D
751,65,96,75,71,74,55,56,99,93,D
752,98,94,97,73,54,77,68,71,78,D
753,85,72,90,73,53,90,100,60,75,D
754,99,58,61,55,95,87,95,88,79,D
...,...,...,...,...,...,...,...,...,...,...
995,78,85,55,84,96,84,63,89,92,D
996,95,98,64,61,92,88,54,83,57,D
997,91,58,87,60,93,65,88,65,55,D
998,79,69,60,62,54,81,68,66,90,D


In [26]:
final_df.to_csv("balanced_dataset.csv", index=False)